## County Health Rankings
Data drawn from census.gov

In [61]:
#Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [62]:
#Import data files
#Set path of input file
main_data = "Resources/combined_ratings.csv"
data_2020 = "Resources/DP03_counties_2020.csv"
data_2015 = "Resources/DP03_counties_2015.csv"

In [63]:
#Read combined ratings data file into a DataFrame
combined_ratings_df = pd.read_csv(main_data)
combined_ratings_df

,FIPS,State,County,Deaths,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% Smokers,...,% 65 and over,% Black,% American Indian & Alaska Native,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural
0,1000,Alabama,NaN,81791.0,9943.0,22,4.9,4.9,10.0,21,...,16.9,26.5,0.7,1.5,0.1,4.4,65.4,1,51.6,41.0
1,1001,Alabama,Autauga,791.0,8129.0,21,4.7,4.7,9.0,18,...,15.6,19.3,0.5,1.2,0.1,3.0,74.3,1,51.4,42.0
2,1003,Alabama,Baldwin,2967.0,7354.0,18,4.2,4.3,8.0,17,...,20.4,8.8,0.8,1.2,0.1,4.6,83.1,1,51.5,42.3
3,1005,Alabama,Barbour,472.0,10254.0,30,5.4,5.2,11.0,22,...,19.4,48.0,0.7,0.5,0.2,4.3,45.6,2,47.2,67.8
4,1007,Alabama,Bibb,471.0,11978.0,19,4.6,4.6,10.0,19,...,16.5,21.1,0.4,0.2,0.1,2.6,74.6,0,46.8,68.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,56037,Wyoming,Sweetwater,532.0,7832.0,15,3.4,3.6,10.0,18,...,12.1,1.1,1.5,1.0,0.2,16.1,79.3,2,48.5,10.9
3189,56039,Wyoming,Teton,109.0,2731.0,12,3.0,3.2,8.0,15,...,15.4,0.6,0.9,1.4,0.2,14.9,81.5,4,48.4,46.4
3190,56041,Wyoming,Uinta,256.0,7331.0,16,3.6,3.7,10.0,17,...,14.1,0.7,1.4,0.5,0.1,9.2,87.4,1,49.3,43.1
3191,56043,Wyoming,Washakie,110.0,6586.0,16,3.6,3.7,7.0,17,...,21.7,0.5,1.8,0.8,0.1,14.1,82.4,0,49.4,36.0


In [64]:
#Read 2020 DP03 data file into a DataFrame
data_2020_df = pd.read_csv(data_2020)
data_2020_df

,NAME,GEO_ID,FIPS,Employed 2020
0,"Autauga County, Alabama",0500000US01001,1001,24580
1,"Baldwin County, Alabama",0500000US01003,1003,98768
2,"Barbour County, Alabama",0500000US01005,1005,8707
3,"Bibb County, Alabama",0500000US01007,1007,8303
4,"Blount County, Alabama",0500000US01009,1009,22836
...,...,...,...,...
3216,"Renville County, Minnesota",0500000US27129,27129,6997
3217,"Roseau County, Minnesota",0500000US27135,27135,8019
3218,"Sherburne County, Minnesota",0500000US27141,27141,52213
3219,"Steele County, Minnesota",0500000US27147,27147,18725


In [65]:
#Read 2015 DP03 data file into a DataFrame
data_2015_df = pd.read_csv(data_2015)
data_2015_df

,NAME,GEO_ID,FIPS,Employed 2015
0,"Childress County, Texas",0500000US48075,48075,2600
1,"Comal County, Texas",0500000US48091,48091,54190
2,"Houston County, Texas",0500000US48225,48225,7694
3,"Navarro County, Texas",0500000US48349,48349,20159
4,"Scurry County, Texas",0500000US48415,48415,7371
...,...,...,...,...
3215,"Charlotte County, Virginia",0500000US51037,51037,5266
3216,"Burleson County, Texas",0500000US48051,48051,7352
3217,"Greensville County, Virginia",0500000US51081,51081,3365
3218,"Thurston County, Washington",0500000US53067,53067,119123


In [66]:
#Merge 2015 employment count into combined ratings dataframe
combined_ratings_df = combined_ratings_df.merge(data_2015_df,on=["FIPS","FIPS"])
combined_ratings_df

,FIPS,State,County,Deaths,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% Smokers,...,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,NAME,GEO_ID,Employed 2015
0,1001,Alabama,Autauga,791.0,8129.0,21,4.7,4.7,9.0,18,...,1.2,0.1,3.0,74.3,1,51.4,42.0,"Autauga County, Alabama",0500000US01001,23986
1,1003,Alabama,Baldwin,2967.0,7354.0,18,4.2,4.3,8.0,17,...,1.2,0.1,4.6,83.1,1,51.5,42.3,"Baldwin County, Alabama",0500000US01003,85953
2,1005,Alabama,Barbour,472.0,10254.0,30,5.4,5.2,11.0,22,...,0.5,0.2,4.3,45.6,2,47.2,67.8,"Barbour County, Alabama",0500000US01005,8597
3,1007,Alabama,Bibb,471.0,11978.0,19,4.6,4.6,10.0,19,...,0.2,0.1,2.6,74.6,0,46.8,68.4,"Bibb County, Alabama",0500000US01007,8294
4,1009,Alabama,Blount,1085.0,11335.0,22,4.9,4.9,8.0,19,...,0.3,0.1,9.6,86.9,2,50.7,90.0,"Blount County, Alabama",0500000US01009,22189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,Wyoming,Sweetwater,532.0,7832.0,15,3.4,3.6,10.0,18,...,1.0,0.2,16.1,79.3,2,48.5,10.9,"Sweetwater County, Wyoming",0500000US56037,23010
3138,56039,Wyoming,Teton,109.0,2731.0,12,3.0,3.2,8.0,15,...,1.4,0.2,14.9,81.5,4,48.4,46.4,"Teton County, Wyoming",0500000US56039,14298
3139,56041,Wyoming,Uinta,256.0,7331.0,16,3.6,3.7,10.0,17,...,0.5,0.1,9.2,87.4,1,49.3,43.1,"Uinta County, Wyoming",0500000US56041,10064
3140,56043,Wyoming,Washakie,110.0,6586.0,16,3.6,3.7,7.0,17,...,0.8,0.1,14.1,82.4,0,49.4,36.0,"Washakie County, Wyoming",0500000US56043,3892


In [67]:
#Merge 2020 employment count into combined ratings dataframe
combined_ratings_df = combined_ratings_df.merge(data_2020_df,on=["FIPS","FIPS"])
combined_ratings_df

,FIPS,State,County,Deaths,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% Smokers,...,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,NAME_x,GEO_ID_x,Employed 2015,NAME_y,GEO_ID_y,Employed 2020
0,1001,Alabama,Autauga,791.0,8129.0,21,4.7,4.7,9.0,18,...,74.3,1,51.4,42.0,"Autauga County, Alabama",0500000US01001,23986,"Autauga County, Alabama",0500000US01001,24580
1,1003,Alabama,Baldwin,2967.0,7354.0,18,4.2,4.3,8.0,17,...,83.1,1,51.5,42.3,"Baldwin County, Alabama",0500000US01003,85953,"Baldwin County, Alabama",0500000US01003,98768
2,1005,Alabama,Barbour,472.0,10254.0,30,5.4,5.2,11.0,22,...,45.6,2,47.2,67.8,"Barbour County, Alabama",0500000US01005,8597,"Barbour County, Alabama",0500000US01005,8707
3,1007,Alabama,Bibb,471.0,11978.0,19,4.6,4.6,10.0,19,...,74.6,0,46.8,68.4,"Bibb County, Alabama",0500000US01007,8294,"Bibb County, Alabama",0500000US01007,8303
4,1009,Alabama,Blount,1085.0,11335.0,22,4.9,4.9,8.0,19,...,86.9,2,50.7,90.0,"Blount County, Alabama",0500000US01009,22189,"Blount County, Alabama",0500000US01009,22836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,56037,Wyoming,Sweetwater,532.0,7832.0,15,3.4,3.6,10.0,18,...,79.3,2,48.5,10.9,"Sweetwater County, Wyoming",0500000US56037,23010,"Sweetwater County, Wyoming",0500000US56037,21572
3137,56039,Wyoming,Teton,109.0,2731.0,12,3.0,3.2,8.0,15,...,81.5,4,48.4,46.4,"Teton County, Wyoming",0500000US56039,14298,"Teton County, Wyoming",0500000US56039,14909
3138,56041,Wyoming,Uinta,256.0,7331.0,16,3.6,3.7,10.0,17,...,87.4,1,49.3,43.1,"Uinta County, Wyoming",0500000US56041,10064,"Uinta County, Wyoming",0500000US56041,9710
3139,56043,Wyoming,Washakie,110.0,6586.0,16,3.6,3.7,7.0,17,...,82.4,0,49.4,36.0,"Washakie County, Wyoming",0500000US56043,3892,"Washakie County, Wyoming",0500000US56043,3841


In [68]:
#Drop extra columns
combined_ratings_df = combined_ratings_df.drop(columns=['NAME_x','NAME_y','GEO_ID_x','GEO_ID_y'])
combined_ratings_df

,FIPS,State,County,Deaths,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% Smokers,...,% American Indian & Alaska Native,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Employed 2015,Employed 2020
0,1001,Alabama,Autauga,791.0,8129.0,21,4.7,4.7,9.0,18,...,0.5,1.2,0.1,3.0,74.3,1,51.4,42.0,23986,24580
1,1003,Alabama,Baldwin,2967.0,7354.0,18,4.2,4.3,8.0,17,...,0.8,1.2,0.1,4.6,83.1,1,51.5,42.3,85953,98768
2,1005,Alabama,Barbour,472.0,10254.0,30,5.4,5.2,11.0,22,...,0.7,0.5,0.2,4.3,45.6,2,47.2,67.8,8597,8707
3,1007,Alabama,Bibb,471.0,11978.0,19,4.6,4.6,10.0,19,...,0.4,0.2,0.1,2.6,74.6,0,46.8,68.4,8294,8303
4,1009,Alabama,Blount,1085.0,11335.0,22,4.9,4.9,8.0,19,...,0.7,0.3,0.1,9.6,86.9,2,50.7,90.0,22189,22836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,56037,Wyoming,Sweetwater,532.0,7832.0,15,3.4,3.6,10.0,18,...,1.5,1.0,0.2,16.1,79.3,2,48.5,10.9,23010,21572
3137,56039,Wyoming,Teton,109.0,2731.0,12,3.0,3.2,8.0,15,...,0.9,1.4,0.2,14.9,81.5,4,48.4,46.4,14298,14909
3138,56041,Wyoming,Uinta,256.0,7331.0,16,3.6,3.7,10.0,17,...,1.4,0.5,0.1,9.2,87.4,1,49.3,43.1,10064,9710
3139,56043,Wyoming,Washakie,110.0,6586.0,16,3.6,3.7,7.0,17,...,1.8,0.8,0.1,14.1,82.4,0,49.4,36.0,3892,3841


In [69]:
#Check for missing values in newly imported employment data
print(f"There are {combined_ratings_df['Employed 2020'].isna().sum()} missing values in the 2020 employment data")
print(f"There are {combined_ratings_df['Employed 2015'].isna().sum()} missing values in the 2015 employment data")

There are 0 missing values in the 2020 employment data
There are 0 missing values in the 2015 employment data


In [70]:
#Calculate employment growth rate from 2015 to 2020
combined_ratings_df['Employment Growth Rate'] = (combined_ratings_df["Employed 2020"]-combined_ratings_df["Employed 2015"])/combined_ratings_df['Employed 2015']*100
combined_ratings_df['Employment Growth Rate']

0        2.476445
1       14.909311
2        1.279516
3        0.108512
4        2.915859
          ...    
3136    -6.249457
3137     4.273325
3138    -3.517488
3139    -1.310380
3140   -12.952438
Name: Employment Growth Rate, Length: 3141, dtype: float64

In [71]:
#Examine employment growth rate field for reasonability
combined_ratings_df["Employment Growth Rate"].describe()

count    3141.000000
mean        3.149131
std        13.393284
min       -41.604676
25%        -1.952006
50%         2.702402
75%         7.458380
max       564.062500
Name: Employment Growth Rate, dtype: float64

In [72]:
#Who has a growth rate greater than 100% The max seems very high
growth_over_100 = combined_ratings_df.loc[combined_ratings_df["Employment Growth Rate"]>=100]
growth_over_100

,FIPS,State,County,Deaths,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% Smokers,...,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Employed 2015,Employed 2020,Employment Growth Rate
538,13309,Georgia,Wheeler,94.0,5428.0,25,4.4,4.2,10.0,24,...,0.3,0.0,5.6,56.0,0,35.3,100.0,1339,2804,109.410007
547,15005,Hawaii,Kalawao,NaN,NaN,15,3.3,3.2,NaN,12,...,8.0,48.9,1.1,26.1,3,53.4,100.0,64,425,564.062500


In [73]:
#Investigate Kalawao, HI. Has a new factory opened there? How big is it?
#According to census.gov and combined ratings file, only 80+ people live there. 64 were employed in 2015. 
#Why are 425 employed in 2020? Check import for errors